In [1]:
##XGB

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import connectorx as cx
import math
import time
import shap
import pickle
import seaborn as sns
import os
import pandas as pd
from sqlalchemy import create_engine
import urllib.parse
import mlflow
from mlflow.models import infer_signature
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score, mean_absolute_error
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score, train_test_split
import statistics
import xgboost as xgb
import plotly.express as px
from sklearn.preprocessing import MultiLabelBinarizer

## Data Preparation

In [23]:
database = 'isgpostgres'
user = 'postgres'
password = '123@NjMpTs'
host = '20.62.75.39'
port = '9030'
encoded_password = urllib.parse.quote(password)
connection_string = "postgresql+psycopg2://" + user + ":" + encoded_password + "@" + host + ":" + port + "/" + database
engine = create_engine(connection_string)

sql_command = """ select * from tm_new_data.prev_cur_model_data_240614_AP where volume>0 and prev_volume > 0  """

# Execute the query and fetch the data
seen_df = pd.read_sql_query(sql_command, engine)
seen_df.shape,seen_df.columns

((37301, 36),
 Index(['isg_osm_id', 'highway_id', 'region', 'roadwaytype_id', 'cur_lanes',
        'cur_lanes_cat', 'cur_maxspeed', 'cur_spd_cat', 'cur_final_place',
        'cur_areatype', 'cur_final_place_encoded', 'cur_areatype_encoded',
        'cur_capacity', 'cur_cap_cat', 'prev_connected_count', 'volume',
        'prev_isg_osm_id_arr', 'prev_highway_id_arr', 'prev_region_arr',
        'prev_roadwaytype_id_arr', 'prev_cur_lanes_arr', 'prev_lanes_cat_arr',
        'prev_maxspeed_arr', 'prev_spd_cat_arr', 'prev_final_place_arr',
        'prev_areatype_arr', 'prev_capacity_arr', 'prev_cap_cat_arr',
        'prev_volume_arr', 'prev_ncc_arr', 'max_prev_ncc', 'availble_prev_vol',
        'availble_ramp_vol', 'availble_main_road_vol', 'prev_vol_filter_arr',
        'prev_volume'],
       dtype='object'))

In [24]:
encoded_feature_lst = ['prev_highway_id_arr', 'prev_region_arr',
                           'prev_roadwaytype_id_arr', 'prev_ncc_arr',
                           'prev_lanes_cat_arr', 'prev_spd_cat_arr',
                           'prev_cap_cat_arr', 'prev_final_place_arr',
                           'prev_areatype_arr']

unencoded_feature_lst =  [
                            'highway_id', 'region', 'roadwaytype_id',
                            'cur_lanes_cat','cur_spd_cat','cur_cap_cat','prev_connected_count',
                            'cur_final_place_encoded', 'cur_areatype_encoded' ,
                            'availble_prev_vol','availble_ramp_vol', 'availble_main_road_vol',
                            'prev_volume'
                         ]

target_feature_lst= ['volume']

In [25]:
features_unencoded_df = seen_df[unencoded_feature_lst]

features_unencoded_df.shape

(37301, 13)

In [26]:
encoded_obj_dict = {}
features_encoded_df = pd.DataFrame()
for col in encoded_feature_lst:
    mlb = MultiLabelBinarizer()
    mlb.fit(seen_df[col])  # Fit the encoder
    seen_one_hot_encoded = mlb.transform(seen_df[col]) 
    seen_temp_df = pd.DataFrame(seen_one_hot_encoded, columns=[f'{col}_{i}' for i in range(1, seen_one_hot_encoded.shape[1] + 1)])
    features_encoded_df = pd.concat([features_encoded_df,seen_temp_df],axis=1)    
    encoded_obj_dict[col] = mlb
    
features_encoded_df.shape

(37301, 59)

In [27]:
for col,i in encoded_obj_dict.items():
    print(col,i.classes_)

prev_highway_id_arr [1 2 3 4 5 6]
prev_region_arr [ 1  2  3  4  5  6  7  8  9 10 11]
prev_roadwaytype_id_arr [ 1  2  3  4  6  7  8 10 11 12 13 14 15 16 17 18 19 20 21]
prev_ncc_arr [1 2 3 4 5]
prev_lanes_cat_arr [1 2]
prev_spd_cat_arr [1 2 3]
prev_cap_cat_arr [0 1 2 3]
prev_final_place_arr ['city' 'hamlet' 'locality' 'neighbourhood' 'town' 'unknown' 'village']
prev_areatype_arr ['Rural' 'Urban']


In [28]:
with open('encoders.pkl','wb') as f:
    pickle.dump(encoded_obj_dict, f)

In [29]:
final_seen_df = pd.concat([features_unencoded_df,features_encoded_df],axis=1)
final_seen_df['volume'] = list(map(lambda x : x ,seen_df['volume']))
final_seen_df.shape

(37301, 73)

## Train Test Split

In [30]:
# X,y = final_seen_df.iloc[:,0:-1],final_seen_df[target_feature_lst]
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# X_train.shape,X_test.shape, y_train.shape, y_test.shape

In [31]:
X,y = final_seen_df.iloc[:,0:-1],final_seen_df[target_feature_lst]
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)
X_train.shape,X_test.shape,X_val.shape

((26110, 72), (5596, 72), (5595, 72))

In [32]:
def geh_check(row):
    current_vol = row['volume']
    next_vol = row['predicted_volume']
    metric_value = float(np.sqrt(2 * np.sum((current_vol - next_vol) ** 2) / np.sum(current_vol + next_vol)))
    return metric_value

In [33]:
def get_geh(y_test, y_pred):
    geh_lst = []
    for i,j in zip(y_test, y_pred):
        row = {'volume':i,'predicted_volume':j}
        geh_lst.append(geh_check(row))
    mean_geh = round(statistics.mean(geh_lst),3)
    return mean_geh,geh_lst

In [34]:
os.environ['MLFLOW_TRACKING_USERNAME'] = 'isgadmin'
os.environ['MLFLOW_TRACKING_PASSWORD'] = 'infosense12@'
tracking_server_uri = "https://ptt2-mlflow.isgsuite.com/"
mlflow.set_tracking_uri(tracking_server_uri)
tracking_uri = mlflow.get_tracking_uri()
print("Tracking URI:", tracking_uri)

Tracking URI: https://ptt2-mlflow.isgsuite.com/


In [35]:
# experiment_name = "TM_New_Approach"
# experiment = mlflow.set_experiment(experiment_name)
# experiment_id = experiment.experiment_id
run_name = 'XGB_FC_1_2_3_input_vol_info'
description = 'previous as input values and current is target'
# Get Experiment Details
# print(f"Experiment_id: {experiment.experiment_id}")
print(f"Run Name: {run_name}")
print(f"Description: {description}")
# print(f"Artifact Location: {experiment.artifact_location}")

Run Name: XGB_FC_1_2_3_input_vol_info
Description: previous as input values and current is target


In [36]:
def get_test_dataset(X_test,y_test,y_pred):
    Test_Data_Df = X_test.copy()
    Test_Data_Df['volume'] = y_test['volume']
    Test_Data_Df['predicted_volume']=y_pred
    Test_Data_Df['predicted_volume']=Test_Data_Df['predicted_volume'].round().astype(int)
    Test_Data_Df['diff']=Test_Data_Df['volume']-Test_Data_Df['predicted_volume']
    Test_Data_Df['geh'] = Test_Data_Df.apply(lambda row: geh_check(row), axis=1)
    Test_Data_Df['geh'] = Test_Data_Df['geh'].round(3)
    return Test_Data_Df

def get_regressor_metrices(y_test,y_pred,rows,cols,geh_threshold):
    mse = round(mean_squared_error(y_test, y_pred),3)
    rmse = round(np.sqrt(mse),3)
    r2 = round(r2_score(y_test, y_pred),3)

    adjusted_r2score =  round((1 - (1 - r2) * (rows - 1) / (rows - cols - 1)),3)
    mae = round(mean_absolute_error(y_test, y_pred),3)
    
    m_geh,geh_lst = get_geh(y_test.values, y_pred)
    geh_error_counts = len(list(filter(lambda x : x>geh_threshold,geh_lst)))
    geh_error_per = round(((geh_error_counts/rows)*100),3)
    
    metrices = {'mse':mse,'rmse':rmse,'r2score':r2,'mae':mae,'adjusted_r2score' : adjusted_r2score,
               'mean_geh':m_geh,'test_data_size':rows,'geh_error_counts':geh_error_counts,'geh_error_percentage':geh_error_per}
    return metrices

def get_feature_importance(xg_reg,X_train):
    explainer = shap.TreeExplainer(xg_reg)
    shap_values = explainer.shap_values(X_train[:500])
    fig = shap.summary_plot(shap_values, X_train[:500], plot_type="bar",show=False)
    return fig

def get_actual_vs_prediction(Test_Data_Df):  
    fig = px.scatter(Test_Data_Df, x='volume', y='predicted_volume')
    fig.add_trace(px.line(Test_Data_Df, x='volume', y='volume').data[0])
    fig.update_layout(xaxis_title='Actual Values', yaxis_title='Predicted Values')
    html_str = fig.to_html(full_html=False)
    return html_str
    with open("Actual_Vs_Prediction_Plot.html", "w") as f:
        f.write(html_str)
    mlflow.log_artifact("Actual_Vs_Prediction_Plot.html", "html")

In [37]:
def log_model_summary(model_log_params):
    try:
        model_data = model_log_params['model_data']
        xg_reg = model_data['model_object']
        X_train = model_data['X_train'] 
        X_test = model_data['X_test']
        y_train = model_data['y_train'] 
        y_test = model_data['y_test'] 
        y_pred = model_data['y_pred']
        geh_threshold = model_log_params['geh_threshold']
        rows = model_log_params['test_size']
        cols = model_log_params['no_of_input_features']

        # Log Parameter
        xgb_params = xg_reg.get_params()
        mlflow.log_params(xgb_params)
        
        # Log test datatset
        Test_Data_Df = get_test_dataset(X_test,y_test,y_pred)
        dataset = mlflow.data.from_pandas(Test_Data_Df, targets="volume", predictions="predicted_volume", name ='test_dataset')
        mlflow.log_input(dataset, context="test")
        
        # Log Metrics
        metrices = get_regressor_metrices(y_test,y_pred,rows,cols,geh_threshold)
        mlflow.log_metrics(metrices)
        model_log_params.pop('model_data')
        # Log tags
        mlflow.set_tags(model_log_params)
    
        # Log Feature Importance
        fig = get_feature_importance(xg_reg,X_train)
        image_path = "Feature_Importance.png"    
        plt.savefig(image_path)
        mlflow.log_artifact(image_path, "images")
    

        
        # Log Actual Vs Prediction Plot
        html_str = get_actual_vs_prediction(Test_Data_Df)
        with open("Actual_Vs_Prediction_Plot.html", "w") as f:
            f.write(html_str)
        mlflow.log_artifact("Actual_Vs_Prediction_Plot.html", "html")
    
    
        # Log Model
        signature = infer_signature(X_test, y_pred)
        mlflow.sklearn.log_model(xg_reg, "model", signature=signature)
        return 0
    except Exception as e:
        return e

In [38]:
with mlflow.start_run(experiment_id="59",run_name = run_name,description=description) as run:

    # Define the XGBoost regressor with the specified parameters
    # Define the XGBoost regressor with the specified parameters
    params= {'subsample': 0.8, 'n_estimators': 2000, 'max_depth':8,'learning_rate': 0.02, 
             'colsample_bytree': 0.8, 'gamma' : 5 , 'tree_method': 'exact', 'alpha': 5, 'lambda': 5,
             'min_child_weight':5,
            'objective': 'count:poisson','base_score': np.log(np.mean(y_train)),
             'early_stopping_rounds': 20, 'eval_metric': ['rmse','mae'],
             'random_state':42
            }
    
    
    # Train the model
    xg_reg = xgb.XGBRegressor(**params)
    eval_set = [(X_train, y_train), (X_val, y_val)]
    xg_reg.fit(X_train, y_train, eval_set=eval_set)    # Make predictions on the test set
    y_pred = xg_reg.predict(X_test)
    y_test['volume'] = round(y_test['volume'],0).astype(int)
    y_pred = np.rint(y_pred).astype(int)

    model_data = {
        'model_object' : xg_reg,
        'X_train' : X_train, 'X_test' : X_test,
        'y_train' : y_train , 'y_test' : y_test, 'y_pred' : y_pred,
        'model_params' : params   
    }
    
    model_log_params ={
        'geh_threshold' : 5,
        'test_size':X_test.shape[0], 'no_of_input_features':X_test.shape[1],
        'input_feature_lst':X_train.columns.tolist(),
        'target_feature_lst': target_feature_lst,
        'Model_Name' : 'XGB',
        'Train Size' : X_train.shape[0],
        'model_data' : model_data
    }
    
    # Log Encoding Objects
    mlflow.log_artifact('encoders.pkl','encoding_object')
        
    # Log Model Summary
    output = log_model_summary(model_log_params)
    print(output)
    mlflow.end_run()  

2024/06/19 15:18:41 WARNING mlflow.utils.git_utils: Failed to import Git (the Git executable is probably not on your PATH), so Git SHA is not available. Error: Failed to initialize: Bad git executable.
The git executable must be specified in one of the following ways:
    - be included in your $PATH
    - be set via $GIT_PYTHON_GIT_EXECUTABLE
    - explicitly set via git.refresh(<full-path-to-git-executable>)

All git commands will error until this is rectified.

This initial message can be silenced or aggravated in the future by setting the
$GIT_PYTHON_REFRESH environment variable. Use one of the following values:
    - quiet|q|silence|s|silent|none|n|0: for no message or exception
    - warn|w|warning|log|l|1: for a warning message (logging level CRITICAL, displayed by default)
    - error|e|exception|raise|r|2: for a raised exception

Example:
    export GIT_PYTHON_REFRESH=quiet



In [ ]:
abcd = get_test_dataset(X_test,y_test,y_pred)
abcd['geh'].mean(),abcd[abcd['geh']>5].shape

(1.3518347033595426, (394, 76))

In [ ]:
rows = X_test.shape[0]
cols = X_test.shape[1]
geh_threshold = 5
metrices = get_regressor_metrices(y_test,y_pred,rows,cols,geh_threshold)
metrices

{'mse': 25340.276,
 'rmse': 159.186,
 'r2score': 0.973,
 'mae': 39.722,
 'adjusted_r2score': 0.973,
 'mean_geh': 1.352,
 'test_data_size': 5596,
 'geh_error_counts': 394,
 'geh_error_percentage': 7.041}

In [ ]:
sql_command = """ select * from tm_new_data.prev_cur_model_data_240614_AP where volume<0 and prev_volume > 0 and (prev_connected_count>1 or max_prev_ncc>1 )"""

# Execute the query and fetch the data
unseen_df = pd.read_sql_query(sql_command, engine)
unseen_df.shape,unseen_df.columns

((14258, 36),
 Index(['isg_osm_id', 'highway_id', 'region', 'roadwaytype_id', 'cur_lanes',
        'cur_lanes_cat', 'cur_maxspeed', 'cur_spd_cat', 'cur_final_place',
        'cur_areatype', 'cur_final_place_encoded', 'cur_areatype_encoded',
        'cur_capacity', 'cur_cap_cat', 'prev_connected_count', 'volume',
        'prev_isg_osm_id_arr', 'prev_highway_id_arr', 'prev_region_arr',
        'prev_roadwaytype_id_arr', 'prev_cur_lanes_arr', 'prev_lanes_cat_arr',
        'prev_maxspeed_arr', 'prev_spd_cat_arr', 'prev_final_place_arr',
        'prev_areatype_arr', 'prev_capacity_arr', 'prev_cap_cat_arr',
        'prev_volume_arr', 'prev_ncc_arr', 'max_prev_ncc', 'availble_prev_vol',
        'availble_ramp_vol', 'availble_main_road_vol', 'prev_vol_filter_arr',
        'prev_volume'],
       dtype='object'))

In [7]:
type(unseen_df.prev_highway_id_arr.iloc[0])

list

In [14]:
unseen_features_unencoded_df = unseen_df[unencoded_feature_lst]
unseen_features_unencoded_df.shape

(14258, 13)

In [15]:
encoded_obj_dict

{'prev_highway_id_arr': MultiLabelBinarizer(),
 'prev_region_arr': MultiLabelBinarizer(),
 'prev_roadwaytype_id_arr': MultiLabelBinarizer(),
 'prev_ncc_arr': MultiLabelBinarizer(),
 'prev_lanes_cat_arr': MultiLabelBinarizer(),
 'prev_spd_cat_arr': MultiLabelBinarizer(),
 'prev_cap_cat_arr': MultiLabelBinarizer(),
 'prev_final_place_arr': MultiLabelBinarizer(),
 'prev_areatype_arr': MultiLabelBinarizer()}

In [16]:
unseen_df[encoded_feature_lst]

,prev_highway_id_arr,prev_region_arr,prev_roadwaytype_id_arr,prev_ncc_arr,prev_lanes_cat_arr,prev_spd_cat_arr,prev_cap_cat_arr,prev_final_place_arr,prev_areatype_arr
0,[5],[11],[10],[3],[2],[1],[2],[city],[Urban]
1,[1],[1],[12],[2],[2],[3],[3],[city],[Urban]
2,"[2, 2]","[1, 1]","[16, 16]","[1, 1]","[1, 1]","[1, 1]","[1, 1]","[city, city]","[Urban, Urban]"
3,"[5, 5]","[1, 1]","[1, 1]","[2, 2]","[2, 2]","[2, 2]","[1, 1]","[town, town]","[Urban, Urban]"
4,[5],[1],[1],[2],[1],[3],[0],[town],[Urban]
...,...,...,...,...,...,...,...,...,...
14253,[5],[11],[1],[1],[2],[1],[1],[city],[Urban]
14254,[3],[10],[10],[2],[2],[3],[2],[town],[Urban]
14255,[5],[6],[1],[2],[1],[1],[0],[village],[Rural]
14256,[5],[9],[1],[2],[1],[2],[0],[town],[Rural]


In [17]:
unseen_features_encoded_df = pd.DataFrame()
for col,i in encoded_obj_dict.items():
    unseen_one_hot_encoded = i.transform(unseen_df[col]) 
    unseen_temp_df = pd.DataFrame(unseen_one_hot_encoded, columns=[f'{col}_{i}' for i in range(1, unseen_one_hot_encoded.shape[1] + 1)])
    unseen_features_encoded_df = pd.concat([unseen_features_encoded_df,unseen_temp_df],axis=1)


In [19]:
final_unseen_df = pd.concat([unseen_features_unencoded_df,unseen_features_encoded_df],axis=1)
final_unseen_df['volume'] = list(map(lambda x : x ,unseen_df['volume']))
final_unseen_df.shape,final_seen_df.shape

(14258, 73)

In [20]:
final_seen_df.columns,final_unseen_df.columns

Index(['highway_id', 'region', 'roadwaytype_id', 'cur_lanes_cat',
       'cur_spd_cat', 'cur_cap_cat', 'prev_connected_count',
       'cur_final_place_encoded', 'cur_areatype_encoded', 'availble_prev_vol',
       'availble_ramp_vol', 'availble_main_road_vol', 'prev_volume',
       'prev_highway_id_arr_1', 'prev_highway_id_arr_2',
       'prev_highway_id_arr_3', 'prev_highway_id_arr_4',
       'prev_highway_id_arr_5', 'prev_highway_id_arr_6', 'prev_region_arr_1',
       'prev_region_arr_2', 'prev_region_arr_3', 'prev_region_arr_4',
       'prev_region_arr_5', 'prev_region_arr_6', 'prev_region_arr_7',
       'prev_region_arr_8', 'prev_region_arr_9', 'prev_region_arr_10',
       'prev_region_arr_11', 'prev_roadwaytype_id_arr_1',
       'prev_roadwaytype_id_arr_2', 'prev_roadwaytype_id_arr_3',
       'prev_roadwaytype_id_arr_4', 'prev_roadwaytype_id_arr_5',
       'prev_roadwaytype_id_arr_6', 'prev_roadwaytype_id_arr_7',
       'prev_roadwaytype_id_arr_8', 'prev_roadwaytype_id_arr_9',
    

### Prepare Data For unseen data

In [21]:
run='b276d4b4122b4f46bde078c2aaa045a4'
logged_model=f'runs:/{run}/model'
loaded_model=mlflow.pyfunc.load_model(logged_model)

MlflowException: Run 'b276d4b4122b4f46bde078c2aaa045a4' not found

In [29]:
unseen_pred = xg_reg.predict(final_unseen_df[X_test.columns.tolist()])

In [30]:
final_unseen_df['predicted_volume'] = unseen_pred
final_unseen_df['predicted_volume'] = final_unseen_df['predicted_volume'].round().astype(int)

In [31]:
final_unseen_df = pd.concat([unseen_df[['isg_osm_id','prev_isg_osm_id_arr']],final_unseen_df],axis=1)

In [32]:
final_unseen_df[final_unseen_df['isg_osm_id'].isin([124370035002,1039216297001,124370035001])][[ 'isg_osm_id','availble_prev_vol',
        'availble_ramp_vol', 'availble_main_road_vol', 'prev_volume','predicted_volume']]

,isg_osm_id,availble_prev_vol,availble_ramp_vol,availble_main_road_vol,prev_volume,predicted_volume
2129,124370035001,1,0,1,2928,2361
2130,124370035002,1,1,0,362,966
4840,1039216297001,1,0,1,2928,1314


In [33]:
# def geh_check_unseen(row):
#     current_vol = row['prev_volume']
#     next_vol = row['predicted_volume']
#     metric_value = float(np.sqrt(2 * np.sum((current_vol - next_vol) ** 2) / np.sum(current_vol + next_vol)))
#     return metric_value

In [34]:
# unseen_df_1['GEH'] = unseen_df_1.apply(geh_check_unseen, axis=1)
# unseen_df_1.shape

In [35]:
# unseen_df_1[unseen_df_1['GEH'] > 25 ].shape